In [1]:
pwd

'/home/euclide/notebooks'

In [4]:
data_dir = '/home/euclide/thesis/MELD-master/MELD-master/data/MELD_Dyadic/train_sent_emo_dya.csv'

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv(data_dir, usecols=['Utterance','Sentiment'])
df

,Utterance,Sentiment
0,also I was the point person on my companys tr...,neutral
1,You mustve had your hands full.,neutral
2,That I did. That I did.,neutral
3,So lets talk a little bit about your duties.,neutral
4,My duties? All right.,positive
...,...,...
12835,"When I first moved to the city, I went out a c...",negative
12836,It made me nuts.,negative
12837,"You guys are messing with me, right?",positive
12838,Yeah.,neutral


In [9]:
# Classes: {'negative': 2, 'positive': 1, 'neutral': 0}
df.loc[df.Sentiment == 'neutral', 'Sentiment'] = 0
df.loc[df.Sentiment == 'positive', 'Sentiment'] = 1
df.loc[df.Sentiment == 'negative', 'Sentiment'] = 2

In [12]:
batch_1 = df
df

,Utterance,Sentiment
0,also I was the point person on my companys tr...,0
1,You mustve had your hands full.,0
2,That I did. That I did.,0
3,So lets talk a little bit about your duties.,0
4,My duties? All right.,1
...,...,...
12835,"When I first moved to the city, I went out a c...",2
12836,It made me nuts.,2
12837,"You guys are messing with me, right?",1
12838,Yeah.,0


In [13]:
# for distillBERT
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# for BERT
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

#load pretrained , model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [17]:
# tokenization
tokenized = batch_1['Utterance'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
#padding
max_len = 0
for i in tokenized.values:
    if len(i)>max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [19]:
padded[0]

array([  101,  2036,  1045,  2001,  1996,  2391,  2711,  2006,  2026,
        2194,  2015,  6653,  2013,  1996,  1047,  2140,  1011,  1019,
        2000, 24665,  1011,  1020,  2291,  1012,   102,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0])

In [20]:
np.array(padded).shape

(12840, 95)

In [21]:
# masking
attention_mask = np.where(padded != 0, 1,0)
attention_mask.shape

(12840, 95)

In [22]:
input_ids = torch.tensor(padded)

attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 3747225600 bytes. Error code 12 (Cannot allocate memory)
